# Interfaces

A lot of the power and extensibility in Julia comes from a collection of informal interfaces. By extending a few specific methods to work for a custom type, objects of that type not only receive those functionalities, but they are also able to be used in other methods that are written to generically build upon those behaviors.

## Iteration

    Required methods		Brief description

    iterate(iter)		        Returns either a tuple of the first item and initial state or nothing if empty
    iterate(iter, state)		Returns either a tuple of the next item and next state or nothing if no items remain


    Important optional methods	    Default definition	    Brief description

    IteratorSize(IterType)	            HasLength()	            One of HasLength(), HasShape{N}(), IsInfinite(), or SizeUnknown() as appropriate
    IteratorEltype(IterType)	    HasEltype()	            Either EltypeUnknown() or HasEltype() as appropriate
    eltype(IterType)	            Any	                    The type of the first entry of the tuple returned by iterate()
    length(iter)	                    (undefined)	            The number of items, if known
    size(iter, [dim])	            (undefined)	            The number of items in each dimension, if known


    Value returned by IteratorSize(IterType)	Required Methods

    HasLength()	                                length(iter)
    HasShape{N}()	                                length(iter) and size(iter, [dim])
    IsInfinite()	                                (none)
    SizeUnknown()	                                (none)


    Value returned by IteratorEltype(IterType)	Required Methods

    HasEltype()	                                eltype(IterType)
    EltypeUnknown()	                                (none)

Sequential iteration is implemented by the iterate function. Instead of mutating objects as they are iterated over, Julia iterators may keep track of the iteration state externally from the object. The return value from iterate is always either a tuple of a value and a state, or nothing if no elements remain. The state object will be passed back to the iterate function on the next iteration and is generally considered an implementation detail private to the iterable object.

Any object that defines this function is iterable and can be used in the many functions that rely upon iteration. It can also be used directly in a for loop since the syntax:

    for i in iter   # or  "for i = iter"
        # body
    end

is translated into:

    next = iterate(iter)
    while next !== nothing
        (i, state) = next
        # body
        next = iterate(iter, state)
    end

A simple example is an iterable sequence of square numbers with a defined length:

In [1]:
struct Squares
    count::Int
end

In [2]:
Base.iterate(S::Squares, state=1) = state > S.count ? nothing : (state*state, state+1)

With only iterate definition, the Squares type is already pretty powerful. We can iterate over all the elements:

In [3]:
for i in Squares(7)
    println(i)
end

1
4
9
16
25
36
49


We can use many of the builtin methods that work with iterables, like in, or mean and std from the Statistics standard library module:

In [4]:
25 in Squares(10)

true

In [5]:
using Statistics
mean(Squares(100))

3383.5

In [6]:
std(Squares(100))

3024.355854282583

There are a few more methods we can extend to give Julia more information about this iterable collection. We know that the elements in a Squares sequence will always be Int. By extending the eltype method, we can give that information to Julia and help it make more specialized code in the more complicated methods. We also know the number of elements in our sequence, so we can extend length, too:

In [7]:
Base.eltype(::Type{Squares}) = Int # Note that this is defined for the type

In [8]:
Base.length(S::Squares) = S.count

Now, when we ask Julia to collect all the elements into an array it can preallocate a Vector{Int} of the right size instead of blindly push!ing each element into a Vector{Any}:

In [9]:
collect(Squares(4))

4-element Array{Int64,1}:
  1
  4
  9
 16

While we can rely upon generic implementations, we can also extend specific methods where we know there is a simpler algorithm. For example, there's a formula to compute the sum of squares, so we can override the generic iterative version with a more performant solution:

In [10]:
Base.sum(S::Squares) = (n = S.count; return n*(n+1)*(2n+1)÷6)

In [11]:
sum(Squares(1803))

1955361914

This is a very common pattern throughout Julia Base: a small set of required methods define an informal interface that enable many fancier behaviors. In some cases, types will want to additionally specialize those extra behaviors when they know a more efficient algorithm can be used in their specific case.

It is also often useful to allow iteration over a collection in reverse order by iterating over Iterators.reverse(iterator). To actually support reverse-order iteration, however, an iterator type T needs to implement iterate for Iterators.Reverse{T}. (Given r::Iterators.Reverse{T}, the underling iterator of type T is r.itr.) In our Squares example, we would implement Iterators.Reverse{Squares} methods:

In [12]:
Base.iterate(rS::Iterators.Reverse{Squares}, state=rS.itr.count) = state < 1 ? nothing : (state*state, state-1)

In [13]:
collect(Iterators.reverse(Squares(4)))

4-element Array{Int64,1}:
 16
  9
  4
  1

## Indexing

    Methods to implement	    Brief description
    getindex(X, i)	            X\[i\], indexed element access
    setindex!(X, v, i)	    X\[i\] = v, indexed assignment
    firstindex(X)	            The first index, used in X\[begin\]
    lastindex(X)	            The last index, used in X\[end\]

For the Squares iterable above, we can easily compute the ith element of the sequence by squaring it. We can expose this as an indexing expression S\[i\]. To opt into this behavior, Squares simply needs to define getindex:

In [14]:
function Base.getindex(S::Squares, i::Int)
    1 <= i <= S.count || throw(BoundsError(S, i))
    return i*i
end

In [15]:
Squares(100)[23]

529

Additionally, to support the syntax S\[begin\] and S\[end\], we must define firstindex and lastindex to specify the first and last valid indices, respectively:

In [16]:
Base.firstindex(S::Squares) = 1

In [17]:
Base.lastindex(S::Squares) = length(S)

In [18]:
Squares(23)[end]

529

Note, though, that the above only defines getindex with one integer index. Indexing with anything other than an Int will throw a MethodError saying that there was no matching method. In order to support indexing with ranges or vectors of Ints, separate methods must be written:

In [19]:
Base.getindex(S::Squares, i::Number) = S[convert(Int, i)]

In [20]:
Base.getindex(S::Squares, I) = [S[i] for i in I]

In [21]:
Squares(10)[[3,4.,5]]

3-element Array{Int64,1}:
  9
 16
 25

While this is starting to support more of the indexing operations supported by some of the builtin types, there's still quite a number of behaviors missing. This Squares sequence is starting to look more and more like a vector as we've added behaviors to it. Instead of defining all these behaviors ourselves, we can officially define it as a subtype of an AbstractArray.

## Abstract Arrays

    Methods to implement		                Brief description

    size(A)		                                Returns a tuple containing the dimensions of A
    getindex(A, i::Int)		                (if IndexLinear) Linear scalar indexing
    getindex(A, I::Vararg{Int, N})		        (if IndexCartesian, where N = ndims(A)) N-dimensional scalar indexing
    setindex!(A, v, i::Int)		                (if IndexLinear) Scalar indexed assignment
    setindex!(A, v, I::Vararg{Int, N})	        (if IndexCartesian, where N = ndims(A)) N-dimensional scalar indexed assignment


    Optional methods	                Default definition	                        Brief description

    IndexStyle(::Type)	                IndexCartesian()	                        Returns either IndexLinear() or IndexCartesian(). See the description below.
    getindex(A, I...)	                defined in terms of scalar getindex	        Multidimensional and nonscalar indexing
    setindex!(A, X, I...)	                defined in terms of scalar setindex!            Multidimensional and nonscalar indexed assignment
    iterate	                                defined in terms of scalar getindex	        Iteration
    length(A)	                        prod(size(A))	                                Number of elements
    similar(A)	                        similar(A, eltype(A), size(A))	                Return a mutable array with the same shape and element type
    similar(A, ::Type{S})	                similar(A, S, size(A))	                        Return a mutable array with the same shape and the specified element type
    similar(A, dims::Dims)	                similar(A, eltype(A), dims)	                Return a mutable array with the same element type and size dims
    similar(A, ::Type{S}, dims::Dims)	Array{S}(undef, dims)	                        Return a mutable array with the specified element type and size


    Non-traditional indices	                Default definition	                Brief description

    axes(A)	                                map(OneTo, size(A))	Return the      AbstractUnitRange of valid indices
    similar(A, ::Type{S}, inds)	        similar(A, S, Base.to_shape(inds))	Return a mutable array with the specified indices inds (see below)
    similar(T::Union{Type,Function}, inds)	T(Base.to_shape(inds))	                Return an array similar to T with the specified indices inds (see below)


If a type is defined as a subtype of AbstractArray, it inherits a very large set of rich behaviors including iteration and multidimensional indexing built on top of single-element access.

A key part in defining an AbstractArray subtype is IndexStyle. Since indexing is such an important part of an array and often occurs in hot loops, it's important to make both indexing and indexed assignment as efficient as possible. Array data structures are typically defined in one of two ways: either it most efficiently accesses its elements using just one index (linear indexing) or it intrinsically accesses the elements with indices specified for every dimension. These two modalities are identified by Julia as IndexLinear() and IndexCartesian(). Converting a linear index to multiple indexing subscripts is typically very expensive, so this provides a traits-based mechanism to enable efficient generic code for all array types.

This distinction determines which scalar indexing methods the type must define. IndexLinear() arrays are simple: just define getindex(A::ArrayType, i::Int). When the array is subsequently indexed with a multidimensional set of indices, the fallback getindex(A::AbstractArray, I...)() efficiently converts the indices into one linear index and then calls the above method. IndexCartesian() arrays, on the other hand, require methods to be defined for each supported dimensionality with ndims(A) Int indices. For example, SparseMatrixCSC from the SparseArrays standard library module, only supports two dimensions, so it just defines getindex(A::SparseMatrixCSC, i::Int, j::Int). The same holds for setindex!.

Returning to the sequence of squares from above, we could instead define it as a subtype of an AbstractArray{Int, 1}:

In [22]:
struct SquaresVector <: AbstractArray{Int, 1}
    count::Int
end

In [23]:
Base.size(S::SquaresVector) = (S.count,)

In [24]:
Base.IndexStyle(::Type{<:SquaresVector}) = IndexLinear()

In [25]:
Base.getindex(S::SquaresVector, i::Int) = i*i

Note that it's very important to specify the two parameters of the AbstractArray; the first defines the eltype, and the second defines the ndims. That supertype and those three methods are all it takes for SquaresVector to be an iterable, indexable, and completely functional array:

In [26]:
s = SquaresVector(4)

4-element SquaresVector:
  1
  4
  9
 16

In [27]:
s[s .> 8]

2-element Array{Int64,1}:
  9
 16

In [28]:
s + s

4-element Array{Int64,1}:
  2
  8
 18
 32

In [29]:
sin.(s)

4-element Array{Float64,1}:
  0.8414709848078965
 -0.7568024953079282
  0.4121184852417566
 -0.2879033166650653

As a more complicated example, let's define our own toy N-dimensional sparse-like array type built on top of Dict:

In [30]:
struct SparseArray{T,N} <: AbstractArray{T,N}
    data::Dict{NTuple{N,Int}, T}
    dims::NTuple{N,Int}
end

In [31]:
SparseArray(::Type{T}, dims::Int...) where {T} = SparseArray(T, dims);

In [32]:
SparseArray(::Type{T}, dims::NTuple{N,Int}) where {T,N} = SparseArray{T,N}(Dict{NTuple{N,Int}, T}(), dims);

In [33]:
Base.size(A::SparseArray) = A.dims

In [34]:
Base.similar(A::SparseArray, ::Type{T}, dims::Dims) where {T} = SparseArray(T, dims)

In [35]:
Base.getindex(A::SparseArray{T,N}, I::Vararg{Int,N}) where {T,N} = get(A.data, I, zero(T))

In [36]:
Base.setindex!(A::SparseArray{T,N}, v, I::Vararg{Int,N}) where {T,N} = (A.data[I] = v)

Notice that this is an IndexCartesian array, so we must manually define getindex and setindex! at the dimensionality of the array. Unlike the SquaresVector, we are able to define setindex!, and so we can mutate the array:

In [37]:
A = SparseArray(Float64, 3, 3)

3×3 SparseArray{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [38]:
fill!(A, 2)

3×3 SparseArray{Float64,2}:
 2.0  2.0  2.0
 2.0  2.0  2.0
 2.0  2.0  2.0

In [39]:
A[:] = 1:length(A); A

3×3 SparseArray{Float64,2}:
 1.0  4.0  7.0
 2.0  5.0  8.0
 3.0  6.0  9.0

The result of indexing an AbstractArray can itself be an array (for instance when indexing by an AbstractRange). The AbstractArray fallback methods use similar to allocate an Array of the appropriate size and element type, which is filled in using the basic indexing method described above. However, when implementing an array wrapper you often want the result to be wrapped as well:

In [40]:
A[1:2,:]

2×3 SparseArray{Float64,2}:
 1.0  4.0  7.0
 2.0  5.0  8.0

In this example it is accomplished by defining Base.similar{T}(A::SparseArray, ::Type{T}, dims::Dims) to create the appropriate wrapped array. (Note that while similar supports 1- and 2-argument forms, in most case you only need to specialize the 3-argument form.) For this to work it's important that SparseArray is mutable (supports setindex!). Defining similar, getindex and setindex! for SparseArray also makes it possible to copy the array:

In [41]:
copy(A)

3×3 SparseArray{Float64,2}:
 1.0  4.0  7.0
 2.0  5.0  8.0
 3.0  6.0  9.0

In addition to all the iterable and indexable methods from above, these types can also interact with each other and use most of the methods defined in Julia Base for AbstractArrays:

In [42]:
A[SquaresVector(3)]

3-element SparseArray{Float64,1}:
 1.0
 4.0
 9.0

In [43]:
sum(A)

45.0

If you are defining an array type that allows non-traditional indexing (indices that start at something other than 1), you should specialize axes. You should also specialize similar so that the dims argument (ordinarily a Dims size-tuple) can accept AbstractUnitRange objects, perhaps range-types Ind of your own design. For more information, see Arrays with custom indices.

## Strided Arrays

    Methods to implement		                Brief description
    strides(A)		                        Return the distance in memory (in number of elements) between adjacent elements in each dimension as a tuple. If A is an AbstractArray{T,0}, this should return an empty tuple.
    Base.unsafe_convert(::Type{Ptr{T}}, A)		Return the native address of an array.

    Optional methods	Default definition	    Brief description
    stride(A, i::Int)	strides(A)\[i\]	        Return the distance in memory (in number of elements) between adjacent elements in dimension k.

A strided array is a subtype of AbstractArray whose entries are stored in memory with fixed strides. Provided the element type of the array is compatible with BLAS, a strided array can utilize BLAS and LAPACK routines for more efficient linear algebra routines. A typical example of a user-defined strided array is one that wraps a standard Array with additional structure.

Warning: do not implement these methods if the underlying storage is not actually strided, as it may lead to incorrect results or segmentation faults.

Here are some examples to demonstrate which type of arrays are strided and which are not:

    1:5   # not strided (there is no storage associated with this array.)
    Vector(1:5)  # is strided with strides (1,)
    A = \[1 5; 2 6; 3 7; 4 8\]  # is strided with strides (1,4)
    V = view(A, 1:2, :)   # is strided with strides (1,4)
    V = view(A, 1:2:3, 1:2)   # is strided with strides (2,4)
    V = view(A, \[1,2,4\], :)   # is not strided, as the spacing between rows is not fixed.

## Customizing broadcasting

    Methods to implement	                                        Brief description
    
    Base.BroadcastStyle(::Type{SrcType}) = SrcStyle()	        Broadcasting behavior of SrcType
    Base.similar(bc::Broadcasted{DestStyle}, ::Type{ElType})	Allocation of output container


    Optional methods	

    Base.BroadcastStyle(::Style1, ::Style2) = Style12()	        Precedence rules for mixing styles
    Base.axes(x)	                                                Declaration of the indices of x, as per axes(x).
    Base.broadcastable(x)	                                        Convert x to an object that has axes and supports indexing

    
    Bypassing default machinery	

    Base.copy(bc::Broadcasted{DestStyle})	                        Custom implementation of broadcast
    Base.copyto!(dest, bc::Broadcasted{DestStyle})	                Custom implementation of broadcast!, specializing on DestStyle
    Base.copyto!(dest::DestType, bc::Broadcasted{Nothing})	        Custom implementation of broadcast!, specializing on DestType
    Base.Broadcast.broadcasted(f, args...)	                        Override the default lazy behavior within a fused expression
    Base.Broadcast.instantiate(bc::Broadcasted{DestStyle})	        Override the computation of the lazy broadcast's axes

Broadcasting is triggered by an explicit call to broadcast or broadcast!, or implicitly by "dot" operations like A .+ b or f.(x, y). Any object that has axes and supports indexing can participate as an argument in broadcasting, and by default the result is stored in an Array. This basic framework is extensible in three major ways:

    Ensuring that all arguments support broadcast
    Selecting an appropriate output array for the given set of arguments
    Selecting an efficient implementation for the given set of arguments

Not all types support axes and indexing, but many are convenient to allow in broadcast. The Base.broadcastable function is called on each argument to broadcast, allowing it to return something different that supports axes and indexing. By default, this is the identity function for all AbstractArrays and Numbers — they already support axes and indexing. For a handful of other types (including but not limited to types themselves, functions, special singletons like missing and nothing, and dates), Base.broadcastable returns the argument wrapped in a Ref to act as a 0-dimensional "scalar" for the purposes of broadcasting. Custom types can similarly specialize Base.broadcastable to define their shape, but they should follow the convention that collect(Base.broadcastable(x)) == collect(x). A notable exception is AbstractString; strings are special-cased to behave as scalars for the purposes of broadcast even though they are iterable collections of their characters (see Strings for more).

The next two steps (selecting the output array and implementation) are dependent upon determining a single answer for a given set of arguments. Broadcast must take all the varied types of its arguments and collapse them down to just one output array and one implementation. Broadcast calls this single answer a "style." Every broadcastable object each has its own preferred style, and a promotion-like system is used to combine these styles into a single answer — the "destination style".

### Broadcast Styles

Base.BroadcastStyle is the abstract type from which all broadcast styles are derived. When used as a function it has two possible forms, unary (single-argument) and binary. The unary variant states that you intend to implement specific broadcasting behavior and/or output type, and do not wish to rely on the default fallback Broadcast.DefaultArrayStyle.

To override these defaults, you can define a custom BroadcastStyle for your object:

    struct MyStyle <: Broadcast.BroadcastStyle end
    Base.BroadcastStyle(::Type{<:MyType}) = MyStyle()

In some cases it might be convenient not to have to define MyStyle, in which case you can leverage one of the general broadcast wrappers:

    - Base.BroadcastStyle(::Type{<:MyType}) = Broadcast.Style{MyType}() can be used for arbitrary types.
    - Base.BroadcastStyle(::Type{<:MyType}) = Broadcast.ArrayStyle{MyType}() is preferred if MyType is an AbstractArray.
    - For AbstractArrays that only support a certain dimensionality, create a subtype of Broadcast.AbstractArrayStyle{N} (see below).

When your broadcast operation involves several arguments, individual argument styles get combined to determine a single DestStyle that controls the type of the output container. For more details, see below.

### Selecting an appropriate output array

The broadcast style is computed for every broadcasting operation to allow for dispatch and specialization. The actual allocation of the result array is handled by similar, using the Broadcasted object as its first argument.

    Base.similar(bc::Broadcasted{DestStyle}, ::Type{ElType})

The fallback definition is

    similar(bc::Broadcasted{DefaultArrayStyle{N}}, ::Type{ElType}) where {N,ElType} =
        similar(Array{ElType}, axes(bc))

However, if needed you can specialize on any or all of these arguments. The final argument bc is a lazy representation of a (potentially fused) broadcast operation, a Broadcasted object. For these purposes, the most important fields of the wrapper are f and args, describing the function and argument list, respectively. Note that the argument list can — and often does — include other nested Broadcasted wrappers.

For a complete example, let's say you have created a type, ArrayAndChar, that stores an array and a single character:

In [44]:
struct ArrayAndChar{T,N} <: AbstractArray{T,N}
    data::Array{T,N}
    char::Char
end
Base.size(A::ArrayAndChar) = size(A.data)
Base.getindex(A::ArrayAndChar{T,N}, inds::Vararg{Int,N}) where {T,N} = A.data[inds...]
Base.setindex!(A::ArrayAndChar{T,N}, val, inds::Vararg{Int,N}) where {T,N} = A.data[inds...] = val
Base.showarg(io::IO, A::ArrayAndChar, toplevel) = print(io, typeof(A), " with char '", A.char, "'")

You might want broadcasting to preserve the char "metadata." First we define

In [45]:
Base.BroadcastStyle(::Type{<:ArrayAndChar}) = Broadcast.ArrayStyle{ArrayAndChar}()

This means we must also define a corresponding similar method:

In [46]:
function Base.similar(bc::Broadcast.Broadcasted{Broadcast.ArrayStyle{ArrayAndChar}}, ::Type{ElType}) where ElType
    # Scan the inputs for the ArrayAndChar:
    A = find_aac(bc)
    # Use the char field of A to create the output
    ArrayAndChar(similar(Array{ElType}, axes(bc)), A.char)
end

"`A = find_aac(As)` returns the first ArrayAndChar among the arguments."
find_aac(bc::Base.Broadcast.Broadcasted) = find_aac(bc.args)
find_aac(args::Tuple) = find_aac(find_aac(args[1]), Base.tail(args))
find_aac(x) = x
find_aac(::Tuple{}) = nothing
find_aac(a::ArrayAndChar, rest) = a
find_aac(::Any, rest) = find_aac(rest)

find_aac (generic function with 6 methods)

From these definitions, one obtains the following behavior:

In [47]:
a = ArrayAndChar([1 2; 3 4], 'x')

2×2 ArrayAndChar{Int64,2} with char 'x':
 1  2
 3  4

In [48]:
a .+ 1

2×2 ArrayAndChar{Int64,2} with char 'x':
 2  3
 4  5

In [49]:
a .+ [5,10]

2×2 ArrayAndChar{Int64,2} with char 'x':
  6   7
 13  14

### Extending broadcast with custom implementations

In general, a broadcast operation is represented by a lazy Broadcasted container that holds onto the function to be applied alongside its arguments. Those arguments may themselves be more nested Broadcasted containers, forming a large expression tree to be evaluated. A nested tree of Broadcasted containers is directly constructed by the implicit dot syntax; 5 .+ 2.*x is transiently represented by Broadcasted(+, 5, Broadcasted(*, 2, x)), for example. This is invisible to users as it is immediately realized through a call to copy, but it is this container that provides the basis for broadcast's extensibility for authors of custom types. The built-in broadcast machinery will then determine the result type and size based upon the arguments, allocate it, and then finally copy the realization of the Broadcasted object into it with a default copyto!(::AbstractArray, ::Broadcasted) method. The built-in fallback broadcast and broadcast! methods similarly construct a transient Broadcasted representation of the operation so they can follow the same codepath. This allows custom array implementations to provide their own copyto! specialization to customize and optimize broadcasting. This is again determined by the computed broadcast style. This is such an important part of the operation that it is stored as the first type parameter of the Broadcasted type, allowing for dispatch and specialization.

For some types, the machinery to "fuse" operations across nested levels of broadcasting is not available or could be done more efficiently incrementally. In such cases, you may need or want to evaluate x .* (x .+ 1) as if it had been written broadcast(*, x, broadcast(+, x, 1)), where the inner operation is evaluated before tackling the outer operation. This sort of eager operation is directly supported by a bit of indirection; instead of directly constructing Broadcasted objects, Julia lowers the fused expression x .* (x .+ 1) to Broadcast.broadcasted(*, x, Broadcast.broadcasted(+, x, 1)). Now, by default, broadcasted just calls the Broadcasted constructor to create the lazy representation of the fused expression tree, but you can choose to override it for a particular combination of function and arguments.

As an example, the builtin AbstractRange objects use this machinery to optimize pieces of broadcasted expressions that can be eagerly evaluated purely in terms of the start, step, and length (or stop) instead of computing every single element. Just like all the other machinery, broadcasted also computes and exposes the combined broadcast style of its arguments, so instead of specializing on broadcasted(f, args...), you can specialize on broadcasted(::DestStyle, f, args...) for any combination of style, function, and arguments.

For example, the following definition supports the negation of ranges:

    broadcasted(::DefaultArrayStyle{1}, ::typeof(-), r::OrdinalRange) = range(-first(r), step=-step(r), length=length(r))

### Extending in-place broadcasting

In-place broadcasting can be supported by defining the appropriate copyto!(dest, bc::Broadcasted) method. Because you might want to specialize either on dest or the specific subtype of bc, to avoid ambiguities between packages we recommend the following convention.

If you wish to specialize on a particular style DestStyle, define a method for

    copyto!(dest, bc::Broadcasted{DestStyle})

Optionally, with this form you can also specialize on the type of dest.

If instead you want to specialize on the destination type DestType without specializing on DestStyle, then you should define a method with the following signature:

    copyto!(dest::DestType, bc::Broadcasted{Nothing})

This leverages a fallback implementation of copyto! that converts the wrapper into a Broadcasted{Nothing}. Consequently, specializing on DestType has lower precedence than methods that specialize on DestStyle.

Similarly, you can completely override out-of-place broadcasting with a copy(::Broadcasted) method.

In [50]:
?copyto!

search: copyto! unsafe_copyto!



```
copyto!(dest, do, src, so, N)
```

Copy `N` elements from collection `src` starting at offset `so`, to array `dest` starting at offset `do`. Return `dest`.

---

```
copyto!(dest::AbstractArray, src) -> dest
```

Copy all elements from collection `src` to array `dest`, whose length must be greater than or equal to the length `n` of `src`. The first `n` elements of `dest` are overwritten, the other elements are left untouched.

# Examples

```jldoctest
julia> x = [1., 0., 3., 0., 5.];

julia> y = zeros(7);

julia> copyto!(y, x);

julia> y
7-element Array{Float64,1}:
 1.0
 0.0
 3.0
 0.0
 5.0
 0.0
 0.0
```

---

```
copyto!(dest, Rdest::CartesianIndices, src, Rsrc::CartesianIndices) -> dest
```

Copy the block of `src` in the range of `Rsrc` to the block of `dest` in the range of `Rdest`. The sizes of the two regions must match.

---

```
copyto!(dest::AbstractMatrix, src::UniformScaling)
```

Copies a [`UniformScaling`](@ref) onto a matrix.

!!! compat "Julia 1.1"
    In Julia 1.0 this method only supported a square destination matrix. Julia 1.1. added support for a rectangular matrix.



#### Working with Broadcasted objects

In order to implement such a copy or copyto!, method, of course, you must work with the Broadcasted wrapper to compute each element. There are two main ways of doing so:

    Broadcast.flatten recomputes the potentially nested operation into a single function and flat list of arguments. You are responsible for implementing the broadcasting shape rules yourself, but this may be helpful in limited situations.

    Iterating over the CartesianIndices of the axes(::Broadcasted) and using indexing with the resulting CartesianIndex object to compute the result.

### Writing binary broadcasting rules

The precedence rules are defined by binary BroadcastStyle calls:

    Base.BroadcastStyle(::Style1, ::Style2) = Style12()

where Style12 is the BroadcastStyle you want to choose for outputs involving arguments of Style1 and Style2. For example,

Base.BroadcastStyle(::Broadcast.Style{Tuple}, ::Broadcast.AbstractArrayStyle{0}) = Broadcast.Style{Tuple}()
indicates that Tuple "wins" over zero-dimensional arrays (the output container will be a tuple). It is worth noting that you do not need to (and should not) define both argument orders of this call; defining one is sufficient no matter what order the user supplies the arguments in.

For AbstractArray types, defining a BroadcastStyle supersedes the fallback choice, Broadcast.DefaultArrayStyle. DefaultArrayStyle and the abstract supertype, AbstractArrayStyle, store the dimensionality as a type parameter to support specialized array types that have fixed dimensionality requirements.

DefaultArrayStyle "loses" to any other AbstractArrayStyle that has been defined because of the following methods:

    BroadcastStyle(a::AbstractArrayStyle{Any}, ::DefaultArrayStyle) = a
    BroadcastStyle(a::AbstractArrayStyle{N}, ::DefaultArrayStyle{N}) where N = a
    BroadcastStyle(a::AbstractArrayStyle{M}, ::DefaultArrayStyle{N}) where {M,N} =
        typeof(a)(_max(Val(M),Val(N)))

You do not need to write binary BroadcastStyle rules unless you want to establish precedence for two or more non-DefaultArrayStyle types.

If your array type does have fixed dimensionality requirements, then you should subtype AbstractArrayStyle. For example, the sparse array code has the following definitions:

    struct SparseVecStyle <: Broadcast.AbstractArrayStyle{1} end
    struct SparseMatStyle <: Broadcast.AbstractArrayStyle{2} end
    Base.BroadcastStyle(::Type{<:SparseVector}) = SparseVecStyle()
    Base.BroadcastStyle(::Type{<:SparseMatrixCSC}) = SparseMatStyle()

Whenever you subtype AbstractArrayStyle, you also need to define rules for combining dimensionalities, by creating a constructor for your style that takes a Val(N) argument. For example:

    SparseVecStyle(::Val{0}) = SparseVecStyle()
    SparseVecStyle(::Val{1}) = SparseVecStyle()
    SparseVecStyle(::Val{2}) = SparseMatStyle()
    SparseVecStyle(::Val{N}) where N = Broadcast.DefaultArrayStyle{N}()

These rules indicate that the combination of a SparseVecStyle with 0- or 1-dimensional arrays yields another SparseVecStyle, that its combination with a 2-dimensional array yields a SparseMatStyle, and anything of higher dimensionality falls back to the dense arbitrary-dimensional framework. These rules allow broadcasting to keep the sparse representation for operations that result in one or two dimensional outputs, but produce an Array for any other dimensionality.